In [2]:
'''
Labels:
1 - Management/Executive
2 - Team meeting
3 - Design discussion
4 - Bug scrub
5 - Code review
6 - 1:1
7 - Daily standup
'''

'\nLabels:\n1 - Management/Executive\n2 - Team meeting\n3 - Design discussion\n4 - Bug scrub\n5 - Code review\n6 - 1:1\n7 - Daily standup\n'

In [3]:
# Imports
import pandas as pd # for dataframes
import json # parse meeting data
import numpy as np # testing classifier accuracy

from sklearn.model_selection import train_test_split # split data
from sklearn.feature_extraction.text import CountVectorizer # one-hot vectors for words
from sklearn.feature_extraction.text import TfidfTransformer # calculate term frequencies
from sklearn.naive_bayes import MultinomialNB # naive-bayes classifier
from sklearn.linear_model import SGDClassifier # SVM classifier
from sklearn.pipeline import Pipeline

import nltk 
from nltk.corpus import stopwords # removing stopwords
from nltk.tokenize import word_tokenize # splitting string into individual words
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# Function to convert list of strings to single string
def listToString(l):
  s = ''
  for word in l:
    if word is not l[len(l)-1]:
        s += word + ' '
    else:
        s += word
  return s

In [7]:
# Load JSON object from file
with open('meeting-data.json') as file:
  j = json.load(file)

# Normalize meetings object (flatten into columns)
data = pd.json_normalize(j['meetings'])
data

,MeetingId,Headline,Date,Time,Label,Attendees,Attendee-count
0,294,Webex Team: Exec meetup,8/3/2022,02:00:00 PM - 03:00:00 PM,1,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",12
1,419,Webex Team: Management meeting,8/5/2022,02:00:00 PM - 03:00:00 PM,1,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",12
2,823,Webex Team: Team meetup,8/2/2022,02:00:00 PM - 03:00:00 PM,2,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10
3,300,Webex Team: Team release,8/4/2022,02:00:00 PM - 03:00:00 PM,2,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10
4,122,Webex Team: Design meeting,8/2/2022,02:00:00 PM - 03:00:00 PM,3,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",6
...,...,...,...,...,...,...,...
685,121,Wireless Team: Daily meetup,8/16/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10
686,194,Wireless Team: Daily updates,8/17/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10
687,192,Wireless Team: Daily updates,8/18/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10
688,237,Wireless Team: Daily meetup,8/19/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10


In [8]:
# Function to remove stopwords from row's description
def remove_stopwords(row):
  return listToString([word for word in word_tokenize(row['Headline']) if word not in stop_words])

In [ ]:
#recommend
def recommend():
  times = ['']
  for row in data:
    times.append(row['time'])
  return listToString(['time conflict' for word in (row['time']) if word in times])


In [ ]:
#new recommend

recommend()
data

NameError: ignored

In [10]:
# Lowercase descriptions
data['Headline'].str.lower()

# Remove stop-words and create new column
stop_words = set(stopwords.words('english'))
data['headline-clean'] = data.apply(lambda row:remove_stopwords(row), axis=1)
data

,MeetingId,Headline,Date,Time,Label,Attendees,Attendee-count,description-clean,headline-clean
0,294,Webex Team: Exec meetup,8/3/2022,02:00:00 PM - 03:00:00 PM,1,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",12,Webex Team : Exec meetup,Webex Team : Exec meetup
1,419,Webex Team: Management meeting,8/5/2022,02:00:00 PM - 03:00:00 PM,1,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",12,Webex Team : Management meeting,Webex Team : Management meeting
2,823,Webex Team: Team meetup,8/2/2022,02:00:00 PM - 03:00:00 PM,2,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Webex Team : Team meetup,Webex Team : Team meetup
3,300,Webex Team: Team release,8/4/2022,02:00:00 PM - 03:00:00 PM,2,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Webex Team : Team release,Webex Team : Team release
4,122,Webex Team: Design meeting,8/2/2022,02:00:00 PM - 03:00:00 PM,3,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",6,Webex Team : Design meeting,Webex Team : Design meeting
...,...,...,...,...,...,...,...,...,...
685,121,Wireless Team: Daily meetup,8/16/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Wireless Team : Daily meetup,Wireless Team : Daily meetup
686,194,Wireless Team: Daily updates,8/17/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Wireless Team : Daily updates,Wireless Team : Daily updates
687,192,Wireless Team: Daily updates,8/18/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Wireless Team : Daily updates,Wireless Team : Daily updates
688,237,Wireless Team: Daily meetup,8/19/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Wireless Team : Daily meetup,Wireless Team : Daily meetup


In [11]:
# Count-vectorize descriptions (one-hot encoding)
count_vec = CountVectorizer()
terms = count_vec.fit_transform(data['headline-clean'])
terms.shape

# Apply TF-IDF (calculates term frequencies)
tfidf_transformer = TfidfTransformer()
terms_tfidf = tfidf_transformer.fit_transform(terms)

# Add to dataframe
data['headline-vectorized'] = terms_tfidf.tocoo().toarray().tolist()
data

,MeetingId,Headline,Date,Time,Label,Attendees,Attendee-count,description-clean,headline-clean,headline-vectorized
0,294,Webex Team: Exec meetup,8/3/2022,02:00:00 PM - 03:00:00 PM,1,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",12,Webex Team : Exec meetup,Webex Team : Exec meetup,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,419,Webex Team: Management meeting,8/5/2022,02:00:00 PM - 03:00:00 PM,1,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",12,Webex Team : Management meeting,Webex Team : Management meeting,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,823,Webex Team: Team meetup,8/2/2022,02:00:00 PM - 03:00:00 PM,2,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Webex Team : Team meetup,Webex Team : Team meetup,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,300,Webex Team: Team release,8/4/2022,02:00:00 PM - 03:00:00 PM,2,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Webex Team : Team release,Webex Team : Team release,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,122,Webex Team: Design meeting,8/2/2022,02:00:00 PM - 03:00:00 PM,3,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",6,Webex Team : Design meeting,Webex Team : Design meeting,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...
685,121,Wireless Team: Daily meetup,8/16/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Wireless Team : Daily meetup,Wireless Team : Daily meetup,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
686,194,Wireless Team: Daily updates,8/17/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Wireless Team : Daily updates,Wireless Team : Daily updates,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
687,192,Wireless Team: Daily updates,8/18/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Wireless Team : Daily updates,Wireless Team : Daily updates,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
688,237,Wireless Team: Daily meetup,8/19/2022,02:00:00 PM - 03:00:00 PM,7,"[Yuraq Merita, Stasia Paolino, Kwadwo Gwydion,...",10,Wireless Team : Daily meetup,Wireless Team : Daily meetup,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [13]:
# Split into data/labels (x/y), then train/test
x = data.drop('Label', axis=1)
y = pd.DataFrame({'label':data['Label']})
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.5)
print("\nx-train:")
print(x_train)
print("\ny-train:")
print(y_train)
print("\nx-test:")
print(x_test)
print("\ny-test:")
print(y_test)


x-train:
     MeetingId                              Headline       Date  \
227        139           Cloud Team: Team discussion  8/22/2022   
102        790  Service Provider Team: Issue tracker  8/10/2022   
161        848        Enterprise Team: Design meetup  8/25/2022   
430        209        Analytics Team: Product design  8/15/2022   
674        967        Wireless Team: Standup meeting   8/5/2022   
..         ...                                   ...        ...   
395        840            Meraki Team: Daily sync-up   8/2/2022   
617        883          Security Team: Daily standup  8/17/2022   
607        695           Security Team: Daily meetup   8/7/2022   
393        232              Meraki Team: 1:1 meeting  8/25/2022   
194        467     Enterprise Team: Everyday meeting   8/8/2022   

                           Time  \
227   02:00:00 PM - 03:00:00 PM   
102   02:00:00 PM - 03:00:00 PM   
161   02:00:00 PM - 03:00:00 PM   
430   02:00:00 PM - 03:00:00 PM   
674   02:0

In [14]:
# Fit Naive-Bayes Classifier
nb = MultinomialNB()
nb.fit(x_train['headline-vectorized'].tolist(), y_train['label'].tolist())

# Predict and check accuracy
preds = nb.predict(x_test['headline-vectorized'].tolist())
print(preds)
np.mean(preds == y_test['label'].tolist())

[7 4 5 7 5 1 1 7 2 2 3 7 3 7 1 6 7 7 3 5 6 5 7 3 7 7 7 1 7 6 3 4 3 5 7 7 7
 7 4 7 5 5 7 7 7 7 7 2 3 6 5 2 5 7 7 7 7 3 4 5 7 7 5 1 7 3 5 7 7 1 2 7 5 5
 3 3 7 3 7 7 7 5 7 1 7 3 7 7 1 7 3 7 7 6 2 4 7 5 7 1 2 1 5 2 2 4 7 1 6 7 7
 6 6 4 7 7 1 7 7 7 5 7 7 7 5 5 5 1 7 7 7 7 4 1 7 6 3 7 7 7 7 6 1 6 1 7 2 7
 7 5 7 5 5 7 7 4 7 2 6 7 7 5 7 5 7 5 3 2 5 4 7 3 7 3 3 1 7 6 5 1 1 1 3 2 7
 2 7 2 7 7 3 7 1 7 7 7 7 1 7 5 4 7 3 5 5 7 2 7 7 7 4 6 5 5 1 3 1 4 7 5 7 7
 7 7 5 2 5 4 5 6 7 7 3 5 5 5 2 1 2 1 7 1 1 5 6 2 7 7 1 3 6 7 5 7 7 4 5 6 1
 3 5 4 7 7 7 5 3 1 4 7 7 1 5 6 7 5 1 6 5 7 6 7 7 7 6 4 3 5 7 7 1 5 2 7 5 4
 2 7 7 6 7 7 6 6 5 4 7 1 4 5 5 3 7 3 4 2 1 5 7 4 3 4 7 5 7 5 7 3 1 7 7 1 7
 1 5 5 7 2 3 5 7 7 7 7 7]


0.855072463768116

In [15]:
# Use alternate pipeline approach
classifier = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
#                     ('clf', MultinomialNB()),
                     ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42)), # SVM classifier
])

# Fit Naive-Bayes Classifier
classifier = classifier.fit(x_train['headline-clean'].tolist(), y_train['label'].tolist())

# Predict and check accuracy
preds = classifier.predict(x_test['headline-clean'].tolist())
print(preds)
np.mean(preds == y_test['label'].tolist())

[2 4 5 6 5 1 1 6 2 2 3 7 3 7 1 6 7 2 3 5 4 5 7 3 7 7 7 1 7 6 3 4 3 5 7 6 7
 7 4 7 5 4 7 6 7 7 6 2 3 6 5 2 5 2 7 7 2 3 4 5 2 7 5 1 7 3 5 7 7 1 2 6 5 5
 3 3 2 3 7 6 6 5 7 1 6 3 7 7 1 7 3 7 6 6 2 4 7 5 4 1 2 1 5 2 2 4 6 1 6 6 7
 6 6 4 6 7 1 7 1 7 5 7 7 7 5 5 5 1 4 7 6 7 4 1 7 6 3 7 7 7 7 6 1 6 1 7 2 6
 7 5 7 5 5 6 7 4 4 2 6 7 7 5 7 5 2 5 3 2 5 4 7 3 7 3 3 1 2 6 5 1 1 1 3 2 7
 2 7 2 7 6 3 6 1 7 6 7 7 1 7 5 4 6 3 5 5 7 2 7 6 7 4 6 4 5 1 3 1 4 7 5 6 7
 2 7 5 2 5 4 5 6 6 7 3 5 5 5 2 1 2 1 7 1 1 5 6 2 6 7 1 3 6 7 5 7 7 4 5 6 1
 3 5 4 1 7 7 5 3 1 4 6 7 1 5 6 7 5 1 6 5 7 6 7 1 2 6 4 3 5 2 7 1 5 2 7 5 4
 2 7 6 6 7 7 6 6 5 4 7 1 4 5 5 3 7 3 4 2 1 5 7 4 3 4 2 5 7 5 7 3 1 6 7 1 7
 1 5 5 7 2 3 5 7 7 6 6 7]


0.9797101449275363

In [31]:
# Recommender
x_test['predicted-label'] = preds
sorted_meetings = x_test.loc[(x_test['Date'] == '8/1/2022')].sort_values(by='predicted-label')

office_limit = 30
in_person = 0
count = 0
for attendee_count in sorted_meetings['Attendee-count']:
  if (in_person + attendee_count > office_limit):
    break
  in_person += attendee_count
  count += 1

in_person_meetings = sorted_meetings.iloc[:count,:]
remote_meetings = sorted_meetings.iloc[count:,:]


In [35]:
in_person_meetings[['MeetingId', 'Headline', 'Date', 'Attendee-count']]

,MeetingId,Headline,Date,Attendee-count
138,209,Enterprise Team: Executive meeting,8/1/2022,12
209,607,Cloud Team: Team meetup,8/1/2022,10


In [36]:
remote_meetings[['MeetingId', 'Headline', 'Date', 'Attendee-count']]

,MeetingId,Headline,Date,Attendee-count
485,824,Routing Team: Product release discussion,8/1/2022,10
278,315,Switching Team: Conference,8/1/2022,10
231,305,Cloud Team: Bug scrub discussion,8/1/2022,5
443,2,Analytics Team: PR #8493,8/1/2022,3
651,645,Wireless Team: 1:1 meeting,8/1/2022,2
513,623,Routing Team: 1:1 with Sara,8/1/2022,2
325,322,Switching Team: Standup meeting,8/1/2022,10
532,648,Routing Team: Daily standup,8/1/2022,10


In [ ]:
# IGNORE: Test of NB/SVM with other dataset
'''
from sklearn.datasets import fetch_20newsgroups
train = fetch_20newsgroups(subset='train', shuffle=True)

count_vec = CountVectorizer()
x_counts = count_vec.fit_transform(train.data)
x_counts.shape

tfidf_transformer = TfidfTransformer()
x_counts_tfidf = tfidf_transformer.fit_transform(x_counts)
x_counts_tfidf.shape

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

type(train.data)

text_clf = text_clf.fit(train.data, train.target)

test = fetch_20newsgroups(subset='test', shuffle=True)
#test_counts = count_vec.fit_transform(test.data)
#test_tfidf = tfidf_transformer.fit_transform(test_counts)
predicted = text_clf.predict(test.data)
np.mean(predicted == test.target)
'''

list